# Object Detection(객체 탐지)
<hr>

- 두 가지 작업(task)을 포함한 것을 `Object Detection`이라고 함
    - Localization(위치결정): 물체(객체)의 `위치 정보`를 찾는 것
        - `위치 정보`: 더 정확하게는 물체(객체)가 있을 것 같은 `경계상자`를 찾는 것
    - Classification(분류): 해당 상자에 포함되어 있는 물체(객체)가 `무엇인지 분류`하는 것
    - Localization과 Classification을 수행하기 위해서 일반적으로 두 단계가 필요
        - 1단계. 물체(객체)가 있을 만한 `경계상자`를 추출
            - `경계상자`: resions of interest, `ROI`(관심 영역)
        - 2단계. 추출된 `ROI`를 이용하여 Localization과 Classification을 수행
    - Object Detection 알고리즘은 크게 두 가지로 구분됨
        - `One Stage Detectors(or Algorithm)`
            - 위 두 단계를 한 번에 수행
            - 알고리즘: SSD(Single Shot Detection), YOLO(You Only Look Once) 등
            - 특징: `two stage detectors`에 비해 상대적으로 연산 속도가 빠르고, 정확도는 낮지만 큰 차이는<br>
            없거나 더 좋은 경우도 있음
            - 참고사항: 대표적인 두 알고리즘 중 SSD가 YOLO를 보완하여 만들었고, SSD가 작은 사이즈의 객체를<br>
            탐지하는 성능이 YOLO보다 우수하다고 알려져 있음
        - `Two Stage Detectors(or Algorithm)`
            - 위 두 단계를 구분하여 수행
            - 알고리즘: `R-CNN` family(R-CNN, Fast R-CNN, Faster R-CNN, `Mask R-CNN`)
                - `R-CNN`: Region-Based Convolutional Neural Network(영역 기반 합성곱 신경망)
                - `Mask R-CNN`: 물체(객체) 탐지도 수행하지만 주 목적은 `Image Segmentation`(이미지 분할)
            - 특징: `one stage detectors`에 비해 상대적으로 연산 속도가 느리고, 정확도는 높다.
- Object Detection은 지도학습과 비지도학습 중 `지도학습`에 해당함
    - Localization과 Classification 이라는 두가지 작업에 해당하기 때문에 각각에 대한 레이블이 필요함

### SSD(Single Shot Detection)
<hr>

- 지도학습의 한 종류이기 때문에 이미지 안에 정답 경계상자, 클래스 정보가 담긴 `정답 이미지`가 존재해야 하며,<br>
알고리즘이 `정답 경계상자`를 찾고, 각 `정답 경계상자`에 속한 객체를 예측해야 함
    - `정답 경계상자`: Ground Truth Bounding box, `GTBB`
- 이미지마다 존재하는 객체의 수, 객체의 위치, 객체의 크기, 객체의 형태가 다르다. 그러므로<br>
아무것도 없는 상태의 이미지에서 위치, 크기, 형태가 각각 다른 `GTBB`를 예측하는 것은 어렵기 때문에<br>
`기본상자`라는 것을 사용
    - `기본상자`: Anchor box or Default box
    - `참고사항`: Faster R-CNN에서 이 `anchor boxes`를 활용한 `RPN(Resion Proposal Network)`방식을 처음 도입,<br>
    이후 SSD에서 `RPN` 단계를 Classification과 `통합하여 수행함`
- Default box(기본상자)
    - 다양한 위치, 크기, 형태의 default boxes를 사용하여 default boxes를 기준점으로 GTBB가 default boxes로부터<br>
    얼마나 떨어져 있는지(`offset` 정도), 그리고 해당 GTBB에 존재하는 객체가 무엇인지를 예측함(두 작업 동시 수행)
        - Localization: `offset` 정도 예측, Classification: 물체(객체)의 `클래스 분류`
    - SSD는 default boxes를 원본 이미지에서 예측하지 않고 `사전학습모형(VGG-16)`과 `추가적인 합성곱 레이어`를 이용하여<br>
    특징 맵(feature map)을 추출하고, 추출된 특징 맵(feature map)에서 default boxes를 예측하여 사용함
        - `사전 학습 모형`: backbone network라고 일컬음
- Feature map(특징 맵)
    - SSD는 Object Detection을 위해 사용하는 `특징 맵의 수가 6개`
        - 1차적으로 VGG-16과 같은 `사전학습모형(backbone network)`을 사용하여 `6개`의 특징 맵을 추출하고,<br>
        그 중 `1개`를 사용
        - `추가적인 합성곱 레이어`를 사용하여 `6개`의 특징 맵을 추출하고, 그 중 `5개`를 사용
        - 결론적으로 `사전학습모형(1개) + 합성곱 레이어(5개) = 6개의 크기가 각각 다른 특징 맵`을 사용하고,<br>
        이를 이용하여 물체(객체)를 탐지하여 다양한 크기의 물체(객체)를 잘 찾을 수 있음(`multi-scale object detection`)
            - `multi-scale object detection`은 SSD의 가장 큰 장점 중 하나
    - <img src="markdown/SSD_architecture.png" width="800">

- Feature map에 Default boxes 설정(예측)
    - 하나의 특징 맵은 격자판(grid)하고 생각할 수 있음
        - 격자판(grid)에는 격자판(grid) 크기 만큼의 `cell`이 존재함
            - e.g., 25 cells exists in 5x5 grid(5x5=25)
    - 각각의 cell에 대하여 `4개 or 6개`의 default boxes를 설정함
        - 위 SSD 구조 이미지에서 `Conv4_3[1], Conv10_2[5], Conv11_2[6]`에서는 `cell 당 4개`의 default boxes를,<br>
        `Conv7[2], Conv8_2[3], Conv9_2[4]`에서는 `cell 당 6개`의 default boxes를 설정(예측)함
            - Conv4_3: 38 x 38 x (`4` x (classes + 4)), `5776` x (classes + 4)
            - Conv7: 19 x 19 x (`6` x (classes + 4)), `2166` x (classes + 4)
            - Conv8_2: 10 x 10 x (`6` x (classes + 4)), `600` x (classes + 4)
            - Conv9_2: 5 x 5 x (`6` x (classes + 4)), `125` x (classes + 4)
            - Conv10_2: 3 x 3 x (`4` x (classes + 4)), `36` x (classes + 4)
            - Conv11_2: 1 x 1 x (`4` x (classes + 4)), `4` x (classes + 4)
            - `(classes + 4)`: 클래스들에 대한 `신뢰도(confidence)` + `오프셋 정보(cx, cy, w, h)`
            - 클래스 당 `8432`개의 default boxes를 설정(예측)하여,<br>
            총 `[B, 8732, 4]` 의 offset(location)과 `[B, 8732, 21]`의 confidence를 갖추게 됨
                - `B`: batch size
    - 각 셀에 대한 default boxes의 설정 방법
        - 보다 다양한 형태의 물체를 찾기 위해 다양한 형태의 default boxes를 사용하며, default boxes의 형태는<br>
        `scaling factor`와 `aspect ratio`에 의해 결점됨
        - Scaling factor
            - default boxes의 `상대적인 크기`를 결정
            - k번째 feature map에 대한 scaling factor는 아래와 같이 계산됨
                - $S_k = S_{min} + \frac{S_{max}-S_{min}}{m-1}(k-1),\quad k\in [1, m]$
                - $S_{min} = 0.2,\ S_{max} = 0.9$
                - $m$은 feature map의 개수를 의미하며 SSD는 총 `6개의 feature map`을<br>
                사용하기 때문에 $m$의 최댓값은 `6`
                    - $k=1$, $S_k$ = 0.2 $(S_{min})$
                    - $k=2$, $S_k$ = 0.34
                    - $k=3$, $S_k$ = 0.48
                    - $k=4$, $S_k$ = 0.62
                    - $k=5$, $S_k$ = 0.76
                    - $k=6$, $S_k$ = 0.9 $(S_{max})$
        - Aspect ratio
            - default boxes의 `가로세로 비율(종횡비)`을 결정
            - cell당 default boxes가 `6개`인 경우
                - $a_r\in\{1,2,3,\frac{1}{2},\frac{1}{3}\}$ -> `5개`
                - $a_r=1$ 일 때, 추가적인 `scaling factor`를 사용: $S'_k = \sqrt {S_k S_{k+1}}$ -> `1개`
            - cell당 default boxes가 `4개`인 경우
                - $a_r\in \{1,2,\frac{1}{2}\}$ -> `3개`
                - $a_r=1$ 일 때, 추가적인 `scaling factor`를 사용: $S'_k = \sqrt {S_k S_{k+1}}$ -> `1개`
        - `k번째` Feature map(grid)에 대해 `1개 cell`에 존재하는 `default boxes`의 크기(형태)는<br>
        아래와 같이 결정됨
            - $(w^a_k,\ h^a_k)=(S_k\sqrt{a_r},\ S_k\frac{1}{\sqrt{a_r}})$
            - $(w^a_k,\ h^a_k)=(S'_k,\ S'_k),\quad\mathsf{where}\ \ S'_k=\sqrt{S_k S_{k+1}}$
            - $w^a_k,\ h^a_k$ = `k번째` feature map(grid)의 `a(종횡비)`가 적용된 default boxes의 `너비와 높이`
            - $S_k$ = `k번째` feature map의 `scaling factor`
            - $a_r$ = default boxes의 `aspect ratio(종횡비)`
            - $S'_k$ = 추가적인 `scaling factor`
        - Default box 가로세로 크기의 모든 경우의 수
            - $k=1$일 때
                - (0.2, 0.2)
                - (0.28, 0.14)
                - (0.35, 0.12)
                - (0.14, 0.28)
                - (0.12, 0.35)
                - (0.26, 0.26)
            - $k=2$일 때
                - (0.34, 0.34)
                - (0.48, 0.24)
                - (0.59, 0.2)
                - (0.24, 0.48)
                - (0.2, 0.59)
                - (0.4, 0.4)
            - $k=3$일 때
                - (0.48, 0.48)
                - (0.68, 0.34)
                - (0.83, 0.28)
                - (0.34, 0.68)
                - (0.28, 0.83)
                - (0.55, 0.55)
            - $k=4$일 때
                - (0.62, 0.62)
                - (0.88, 0.44)
                - (1.07, 0.36)
                - (0.44, 0.88)
                - (0.36, 1.07)
                - (0.69, 0.69)
            - $k=5$일 때
                - (0.76, 0.76)
                - (1.07, 0.54)
                - (1.32, 0.44)
                - (0.54, 1.07)
                - (0.44, 1.32)
                - (0.83, 0.83)
            - $k=6$일 때
                - (0.9, 0.9)
                - (1.27, 0.64)
                - (1.56, 0.52)
                - (0.64, 1.27)
                - (0.52, 1.56)
                - (0.97, 0.97)
        - 각 cell마다 default boxes가 위치할 `중심점` 설정
            - 각각의 cell마다 4개 or 6개의 default boxes가 위치하려면 `해당 cell의 중심점`이 필요하며,<br>
            그 중심점은 아래와 같이 계산됨
            - $\left(\frac{i+0.5}{|f_{k}|},\ \frac{j+0.5}{|f_{k}|}\right),\quad{\scriptstyle\text{where }}|f_{k}|={\scriptstyle\text{k번째 square feature map의 크기}},\quad i,j\in[0,|f_{k}|)$

- Training(학습하기)
    - 생성된 각각의 default box가 하나의 `관측치`로 간주되며, 각 `관측치`마다 그에 대한<br>
    `비용함수(loss function)` 계산이 필요함
        - 각 `관측치`에 대해 `비용함수`를 계산하기 위해서는 각 `관측치`에 대해 `정답상자(GTBB)`를 `할당(assign)`해야 함
            - 각 `관측치`에 대해 `GTBB`를 `할당`하기 위해 SSD에서 사용된 `matching strategy`
                - 각 `관측치`는 자기 자신에 대한 `offset`값과 각 클래스에 대한 `confidence`값을 가지고 있고,<br>해당 `offset`값을 이용하여 `GTBB`와 겹치는 정도를 계산하기 위해 `Jaccard overlap` 지표를 사용함
                    - `Jaccard overlap`: object detector의 정확도를 측정하는데 이용되는 평가 지표,<br>
                    `Intersection over Union(IoU)`라고도 칭하며, `0~1` 사이의 실수값을 가짐
                    - <img src="markdown/intersection_over_union.png" width="400">
                - 각 `GTBB`에 대하여 `IoU값`이 가장 큰 `관측치`를 찾고, 해당 `GTBB`를 해당 `관측치`의<br>
                정답으로 `할당`
                - 이후 `GTBB`를 할당 받지 못 한 `관측치` 중 `IoU값`이 `0.5`보다 큰 `관측치` 또한 해당 `GTBB`를<br>
                정답으로 `할당` 받음
                    - 만약 임의의 `관측치`에서 `IoU값`이 `0.5`보다 큰 `GTBB`가 여러개 존재한다면 그 중 가장 큰<br>
                    `IoU값`을 갖는 `GTBB`를 정답으로 할당 받음
                - 할당 작업이 모두 끝나면 `GTBB`를 할당 받은 `관측치`와 그렇지 않은 `관측치`를 구분하기위해<br>
                `GTBB`가 할당된 `관측치`를 `positive box`라고 하고,<br>
                그렇지 않은 `관측치`를 `negative box`라고 분류함

In [30]:
from math import *
k = 5
S = [0.2,0.34,0.48,0.62,0.76,0.9,1.04]
a = [1,2,3,1/2,1/3]
for i in range(6):
    if i == 5:
        print((round(sqrt(S[k]*S[k+1]),2), round(sqrt(S[k]*S[k+1]),2)))
        break
    print((round(S[k]*sqrt(a[i]),2), round(S[k]*(1/sqrt(a[i])),2)))

(0.9, 0.9)
(1.27, 0.64)
(1.56, 0.52)
(0.64, 1.27)
(0.52, 1.56)
(0.97, 0.97)


In [31]:
import math


S_MIN = 0.2
S_MAX = 0.9
def get_scales(m):
    scales = []    
    for k in range(1, m+1):
        scales.append(round((S_MIN + (S_MAX - S_MIN) / (m - 1) * (k - 1)), 2)) 
    return scales
get_scales(6)
((()))
RATIOS = [1, 2, 3, 0.5, 0.33]
def get_width_height(scales):
    width_heights = []
    for k, scale in enumerate(scales):
        print(f'k: {k+1} scale: {scale}')
        width_height_per_scale = []
        for ratio in RATIOS:
            width = min(round((scale *  math.sqrt(ratio)), 2), 1)
            height = min(round((scale /  math.sqrt(ratio)), 2), 1)
            width_height_per_scale.append((width, height))
            print(f'widht: {width} height: {height}')
        if k < len(scales) - 1:
            extra_sacle = round(math.sqrt(scale * scales[k+1]), 2)
            width_height_per_scale.append((extra_sacle, extra_sacle))
            print(f'width: {extra_sacle} height: {extra_sacle}')
        width_heights.append(width_height_per_scale)
        print('')
    return width_heights

scales = get_scales(6)
width_heights = get_width_height(scales)

k: 1 scale: 0.2
widht: 0.2 height: 0.2
widht: 0.28 height: 0.14
widht: 0.35 height: 0.12
widht: 0.14 height: 0.28
widht: 0.11 height: 0.35
width: 0.26 height: 0.26

k: 2 scale: 0.34
widht: 0.34 height: 0.34
widht: 0.48 height: 0.24
widht: 0.59 height: 0.2
widht: 0.24 height: 0.48
widht: 0.2 height: 0.59
width: 0.4 height: 0.4

k: 3 scale: 0.48
widht: 0.48 height: 0.48
widht: 0.68 height: 0.34
widht: 0.83 height: 0.28
widht: 0.34 height: 0.68
widht: 0.28 height: 0.84
width: 0.55 height: 0.55

k: 4 scale: 0.62
widht: 0.62 height: 0.62
widht: 0.88 height: 0.44
widht: 1 height: 0.36
widht: 0.44 height: 0.88
widht: 0.36 height: 1
width: 0.69 height: 0.69

k: 5 scale: 0.76
widht: 0.76 height: 0.76
widht: 1 height: 0.54
widht: 1 height: 0.44
widht: 0.54 height: 1
widht: 0.44 height: 1
width: 0.83 height: 0.83

k: 6 scale: 0.9
widht: 0.9 height: 0.9
widht: 1 height: 0.64
widht: 1 height: 0.52
widht: 0.64 height: 1
widht: 0.52 height: 1

